<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [1]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [ ]:
# Veuillez écrire votre code ici.


In [ ]:
from spacy.lang.fr.examples import sentences
s1 = sentences[2] # prenons la 3e phrase comme exemple

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [ ]:
# Veuillez écrire votre code et votre réponse ici.


**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`. 

In [ ]:
# Veuillez écrire votre code ici.


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

In [73]:
# Les informations sur les relations de dépendance sont dans la 8ème colonne. Elles sont représentées par le nom de la relation de dépendance.
# Example : "nsubj" pour "nom sujet", "ROOT" pour le noeud racine, "det" pour "déterminant", "amod" pour "adjectif modificateur", "punct" pour "ponctuation", etc.

In [74]:
from spacy.tokens import DocBin, Doc
test_data = DocBin().from_disk("../Labo2/spacy_data/fr-ud-test.spacy")
# for doc in test_data.get_docs(nlp.vocab):  # exemple
#     for sent in doc.sents:
#         print(sent)

**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [104]:
docs = list(test_data.get_docs(nlp.vocab))
if len(docs) > 1:
    doc2 = list(docs[1].sents)
    if len(doc2) > 6:
        s2 = doc2[6]
        print(s2.text)

Trois ans plus tard, il tient un discours sur la crise.


**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [76]:
from spacy import displacy
from IPython.display import HTML, display


In [100]:
display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(s2, style="dep", jupyter=False)
)))

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [101]:
nlpS2 = nlp(s2.text)

display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(nlpS2, style="dep", jupyter=False)
)))

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

In [102]:
# Comparer les structures de dépendances
correct_deps = 0
total_deps = 0

s2_deps = [(token.i, token.dep_, token.head.i) for token in s2]
nlpS2_deps = [(token.i, token.dep_, token.head.i) for token in nlpS2]

# Afficher les dépendances côte à côte pour comparaison
print("Original vs. Predicted")
print("=======================")
for i, (orig, pred) in enumerate(zip(s2_deps, nlpS2_deps)):
    orig_token = s2[i].text
    is_correct = orig[1] == pred[1] and (orig[2] - 161) == pred[2]
    
    if is_correct:
        correct_deps += 1
    total_deps += 1
    
    print(f"{orig_token}: {orig[1]}->{s2[(orig[2] - 161)].text if (orig[2] - 161) < len(s2) else 'ROOT'} vs {pred[1]}->{nlpS2[pred[2]].text if pred[2] < len(nlpS2) else 'ROOT'} {'✓' if is_correct else '✗'}")

# Calculer le taux de correction
accuracy = correct_deps / total_deps if total_deps > 0 else 0
print(f"\nTaux de correction: {accuracy:.3f} ({correct_deps}/{total_deps})")




Original vs. Predicted
Trois: nummod->ans vs nummod->ans ✓
ans: obl->tient vs obl:mod->plus ✗
plus: advmod->tard vs advmod->tard ✓
tard: advmod->ans vs ROOT->tard ✗
,: punct->tient vs punct->tient ✓
il: nsubj->tient vs nsubj->tient ✓
tient: ROOT->tient vs ROOT->tient ✓
un: det->discours vs det->discours ✓
discours: obj->tient vs obj->tient ✓
sur: case->crise vs case->crise ✓
la: det->crise vs det->crise ✓
crise: nmod->discours vs nmod->discours ✓
.: punct->tient vs punct->tient ✓

Taux de correction: 0.846 (11/13)


**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [84]:
from spacy.scorer import Scorer
from spacy.training import Example

In [117]:
# Créer un scorer et une exemple pour l'évaluation
scorer = Scorer()

# Créer un exemple pour l'évaluation
example = Example(nlpS2, s2.as_doc())

# Calculer les scores (scorer.score prend une liste d'exemples)
scores = scorer.score([example])

# Afficher les scores des dépendances avec trois décimales
print(f"UAS: {scores['dep_uas']:.3f}")  # Unlabeled Attachment Score
print(f"LAS: {scores['dep_las']:.3f}")  # Labeled Attachment Score

# Comparer avec les résultats de la question précédente
# La justification est imprimée pour expliquer la différence/similarité
print("\nComparaison avec le taux de correction précédent:")
print(f"Taux précédent: {correct_deps / total_deps:.3f}")
print("La différence entre les scores peut être expliquée par la méthode de comparaison.")
print("Dans la question précédente, le score inclut la comparaison de la ponctuation. Le score UAS/LAS ne le fait pas.")
print(f"Si on exlut la ponctuation, le score est le meme: {(correct_deps - 2) / (total_deps - 2):.3f}")

UAS: 0.818
LAS: 0.818

Comparaison avec le taux de correction précédent:
Taux précédent: 0.846
La différence entre les scores peut être expliquée par la méthode de comparaison.
Dans la question précédente, le score inclut la comparaison de la ponctuation. Le score UAS/LAS ne le fait pas.
Si on exlut la ponctuation, le score est le meme: 0.818


## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [ ]:
# Veuillez écrire votre code ici.


**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [ ]:
# Veuillez écrire votre code ici.


## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [ ]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.
!python -m spacy train config.cfg \
  --output ./myDEPparser1 \
  --paths.train ../Labo2/spacy_data/fr-ud-train.spacy \
  --paths.dev ../Labo2/spacy_data/fr-ud-dev.spacy \
  --verbose

In [ ]:
# Veuillez indiquer ici le nombre d'époques final.


**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [ ]:
# Veuillez écrire votre code ici.


**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [ ]:
# Veuillez écrire votre code ici.


**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.